- called remote dictionary store
- its a key value data store
- used in gameing due to its fast performance
- 2 aws services
  - amazon memorydb for redis
    - redis compatible in memory database service
  - amazon elasticache for redis
    - in memory database service
- elasticache is for in-memory data store
- big o notation the o stands for order of magnitude
# Elasticache
- its fully managed
- fully managed service
  - no need to manage the underlying infrastructure
- extremely fast
- engines
  - accepts memcached
    - works by LRU
    - key value store
    - input format "set key flags expiry bytes"
  - redis
    - several accepted data structures
    - accepts asynchrnous operations\
    - scalable     
# why need in-memory data store
- memory is 50x faster
- no need for query processing overhead
- its key based
- people cannot tolerate waiting for a few seconds
- networks can bottleneck speeds
# redis
- start a redist docker container to work with
  - `docker run -p 6379:6379 -d --name myredis redis`
- databases store data in an inert form where as redis does not
- the data can stored as an kept encoded
  - encoding is to transform data into something easier to transfer
- caching
  - improves read performance
  - if calling directly from the database the reads and writes compete for the same resources
  - can cache s3 objects
  - successfully retrieving data from the cache is called a cache hit
    - otherwise its a cache miss
      - data could have been evicted  
  - use cases
    - lazy loading
      - asks for data from cache
        - if cache miss
          - asks for data from database
              - return the data
                - store the data in the cache
        - if cache hit
          - return the data
      - advantages
        - constantly repopulated
        - avoids unecessary data in cache
      - disadvantages
        - misses can be expensive
        - fresh data will make it less likely for cache to store it
    - ask for website data from cache
- session store
  - storing tokens, shopping carts, and other user data
  - hash can store user info
    - can edit without changing whole object
- leaderboards
  - redis sorts data on insertion
  - O(log n) time complexity
- geospatial
  - lets you store coordinates avia geospatial indexes
  - useful for finding nearby points
- message queues
  - pushes tasks to another server via a queue  
- machine learning
  - submillisecond access to models
- media streaming
  - authentication
  - available shows
- real-time analytics
  - bidding
  - data speeds exceed disk writing
  - data deduplication
    - remove duplicate data
- chatt apps
  - create a nabagenebt system
  - selective subscriptions
- data structures
  - geospatial = set of coords
  - hyperloglog
    - cardinality
      - number of relationships between rows of one table with another
- data types
  - string and hashing are the most commonly used
  - string
    - `set key value`
  - hash
    - `hmset userid:seb first Seb team Kuoly`
  - list
    - doubly linked
      - can push easily to the start and end
    - `rpush todo "first" "second"`
  - set
    - unique unordered string values
    - good for joining
    - `sadd myset "first" "second"`
  - leaderboard
    - `zadd leaderboard 1000 seb 2000 kyle`
  - geospacial
    - based on unsorted sets
    - `geoadd cities 0 0 "London"`
  - hyperloglog
    - meant to estimate cardinality
    - has a max of 12k memory
    - exceptionally fast
    - does not store values
    - used for visitor count or ip addresses
    - `pfadd pfadd "first" "second"`
# caching patterns

In [60]:
import redis
r = redis.Redis()

In [61]:
# hash
r.mset({"Bahamas":"Nassau"})
r.get("Bahamas")
# string
r.set("Bahamas", "Nassau")
r.get("Bahamas")
# sorted set
r.zadd("countries", {"Bahamas": 1, "Canada": 2})
r.zrevrange("countries", 0, -1)

[b'Canada', b'Bahamas']

In [62]:
# create a fake db
data = {
  "countries": {
    "Armenia": "AM",
    "Aruba": "AW",
    "Australia": "AU",
    "Austria": "AT"
  }
}
data["countries"]

{'Armenia': 'AM', 'Aruba': 'AW', 'Australia': 'AU', 'Austria': 'AT'}

- lazy loading

In [63]:
# lazy loading
def lazy_loading(country):
    r_res = r.get(country)
    if r_res:
        r_res = r_res.decode("utf-8")
        print(f"{country} - cache hit")
        return r_res
    else:
        print(f"{country} - cache miss")
        res = data["countries"][country]
        r.setex(country, 10, res)
        return res
lazy_loading("Australia")

Australia - cache miss


'AU'

- write through
  - proactivly populating cache as opposed to reactlively
  - dynabo DB triggers a lambda function on modification
    - update redis
  
  - advantages
    - data is never stale
    - write latency is better tolerated by customers
  - disadvanrages
    - unecessary cached data

In [64]:
data_user = {
    "users": {
        "userid:seb": {
            "name": "Sebastian",
            "age": 22,
        }
    }
}
r.hset("userid:seb", mapping=data_user["users"]["userid:seb"])
r.hget("userid:seb", "age")

b'22'

In [65]:
# update country code
def write_through(user, age):
    data_user["users"][user]["age"] = age
    r.hset(user, "age", age)
write_through("userid:seb", 23)
r.hget("userid:seb", "age")


b'23'

- you can create redis key pairs with hashed key
  - key is a hashed sql query
  - value is the result of the query
# evictions
- lru
  - keys evicted to when they were last used
- lfu
  - approximation of least used
- volatile
  - only keys with an expiry are evicted
  - the rest will be sotred untill they are actively evicted
- ttl
  - time to live
  - must be explicitly assigned
  - can be timestamped `exporeat`
- expiration storms
  - many keys evicted simultaneously
# clients
- connection pools
- exponential back-off
  - prevent all customers from reconnecting at the same time
# pub-sub
- focuses on simplicity of implementation
- solves communication between recievers and creators of messages
- redis serves as the central messaging system
- fire and forget messaging
  - once message is sent it is not stored
- subscribe 
  - `psubscribe somekey:*`
- publish
  - `publish somekey:1 "hello world"`
- clients must only subscribe to the channels they need to

# streams
- it is a data structure
- high velocity analytics
- data can come in at different speeds
- can scale horizontally and verically
- wont get overwhelmed by data
- worker nodes can pull entries from the stream and process them
  - could be assigned as autoscaling groups
- consumer
  - `xread block 0 streams someserver:b $`
- producer
  - `xadd someserver:a * fan:4:rpm 1500 fan:5:rpm 2000`
- data is persisted into a datascructure
# global data store
- has an active cluster that recieves reads and writes
- clusters in other regions read from the active cluster
- can failover to secondary clusters
  - promotest the secondary cluster
- does not work for Pub/Sub
  - pub/sub remains regional